In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


base_model = torch.load('./models/Qwen3-8B-AWQ-SVD-MMLU-fft/model_finetuned.pt', weights_only=False)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-8B", trust_remote_code=True)

/home/timmiakov/internal_exps/qwentization/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/timmiakov/internal_exps/qwentization/.venv/lib/python3.10/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hanse

In [2]:
base_model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
          (k_proj): WQLinear_GEMM(in_features=4096, out_features=1024, bias=False, w_bit=4, group_size=128)
          (v_proj): WQLinear_GEMM(in_features=4096, out_features=1024, bias=False, w_bit=4, group_size=128)
          (o_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): WQLinear_GEMM(in_features=4096, out_features=12288, bias=False, w_bit=4, group_size=128)
          (up_proj): WQLinear_GEMM(in_features=4096, out_features=12288, bias=False, w_bit=4, group_size=128)
          (down_proj)

In [3]:
from src.mmlu_benchmark import MMLUEvaluator

evaluator = MMLUEvaluator(
    model=base_model, tokenizer=tokenizer, device="cuda",
    split="dev", per_subject_samples=10, seed=42, model_name="awq_mmlu"
)

_ = evaluator.evaluate()

  Загружена dev выборка
  Всего вопросов в dev выборке: 285
  Количество предметов: 57
Инициализация завершена. Эксперимент: awq_mmlu_dev_20251223_190245

Эксперимент: awq_mmlu_dev_20251223_190245
Модель: awq_mmlu
Всего вопросов в dev: 285
Количество предметов: 57
Промпт стиль: zero-shot


57it [01:19,  1.40s/it]

ОБЩАЯ ТОЧНОСТЬ: 0.4070 (40.70%)
Правильных ответов: 116 из 285
Оценено предметов: 57
Пиковое потребление VRAM: 8119.99 MB
